In [1]:
#данные уже готовы

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import make_scorer
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

import data_manager as dm

import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# first_popularity_steel = pd.read_csv('first_popularity_steel.csv', sep=',')
# second_popularity_steel = pd.read_csv('second_popularity_steel.csv', sep=',')
# инициализируйте ваш датасет и идите вниз

In [3]:
#first_popularity_steel.info()

In [4]:
#second_popularity_steel.info()

In [5]:
def PRE_PREDICT(X, y, first_to_predict, last_target):
    print("PRE_PREDICT")
    
    for target in first_to_predict:
        y_target = y[target]
        score = cross_val_score(CatBoostRegressor(iterations=1000,
                                                  learning_rate=0.1,
                                                  depth=5,
                                                  verbose=False,
                                                  random_seed=42), X, y_target, scoring=make_scorer(mean_absolute_percentage_error), cv=5).mean()
        print(target, "\n", score * 100, "%")
        predict = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, y_target).predict(X)
        X[target] = predict
    #
    print("TARGET: ", last_target)
    y_target = y[last_target]
    score = cross_val_score(CatBoostRegressor(iterations=1000,
                                                  learning_rate=0.1,
                                                  depth=5,
                                                  verbose=False,
                                                  random_seed=42), X, y_target, scoring=make_scorer(mean_absolute_percentage_error), cv=5).mean()
    print(score * 100, "%")
    predict = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, y_target).predict(X)    
    print("END")
    
    return predict

In [6]:
# now we know what to do and now we know what to hack!

def LEARN_TO_COOK_SLAG(data):
    to_drop = ['химшлак последний Al2O3', 'химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R',
       'химшлак последний SiO2']

    X = data.drop(to_drop, axis=1)
    data_for_dirt = data[['химшлак последний R', 'химшлак последний SiO2', 'химшлак последний CaO']]
    df_for_dirt = pd.DataFrame()
    
    df_for_dirt['химшлак последний CaO'] = PRE_PREDICT(X.copy(), data_for_dirt, ['химшлак последний R', 'химшлак последний SiO2'], 'химшлак последний CaO')
    df_for_dirt['химшлак последний SiO2'] = PRE_PREDICT(X.copy(), data_for_dirt, ['химшлак последний R', 'химшлак последний CaO'], 'химшлак последний SiO2')
    df_for_dirt['химшлак последний R'] = PRE_PREDICT(X.copy(),data_for_dirt, ['химшлак последний CaO', 'химшлак последний SiO2'], 'химшлак последний R')
    
    X['химшлак последний CaO_1'] = df_for_dirt['химшлак последний CaO']
    X['химшлак последний SiO2_1'] = df_for_dirt['химшлак последний SiO2']
    X['химшлак последний R_1'] = df_for_dirt['химшлак последний R']
    
    print("PREPROCESS NEXT STEP")

    targets = ['химшлак последний FeO', 'химшлак последний MgO',
           'химшлак последний MnO', 'химшлак последний Al2O3']

    for i in range(len(targets)):
        best_score_target = ""
        best_score = 1
        best_target_prediction = []
    
        for target in targets:
            y = data[target] 
            reg = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42)
            score = cross_val_score(reg, X, y, scoring=make_scorer(mean_absolute_percentage_error), cv=5).mean()      

            if score < best_score:
                best_score_target = target
                best_score = score
        best_target_prediction = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, data[best_score_target]).predict(X)

        print(best_score_target, "\n", best_score * 100, "%")
        targets.remove(best_score_target)
        X[best_score_target + "_1"] = best_target_prediction
        
    print("END PREPROCESS")
        
    targets = ['химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R', 'химшлак последний Al2O3', 'химшлак последний SiO2']
    
    for i in range(len(targets)):
        best_score_target = ""
        best_score = 1
        best_target_prediction = []
    
        for target in targets:
            y = data[target] 
            reg = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42)
            score = cross_val_score(reg, X, y, scoring=make_scorer(mean_absolute_percentage_error), cv=5).mean()      

            if score < best_score:
                best_score_target = target
                best_score = score
        best_target_prediction = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, data[best_score_target]).predict(X)

        print(best_score_target, "\n", best_score * 100, "%")
        targets.remove(best_score_target)
        X[best_score_target] = best_target_prediction
    

In [7]:
df_init = dm.load_data('evraz_data_prprocessed_Anvar.csv')

In [8]:
df_init.s

In [12]:
LEARN_TO_COOK_SLAG(df_init) # подставьте ваш датасет

PRE_PREDICT
химшлак последний R 
 nan %


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="Э76ХАФ": Cannot convert 'b'\xd0\xad76\xd0\xa5\xd0\x90\xd0\xa4'' to float